In [ ]:
import requests
import json
import pandas as pd
import getpass

ig_account = input("Enter IG account:")
pass_ig_account = getpass.getpass('Enter Password:')
ig_api_key = input("Enter IG API key:")

#prod server
#url = 'https://api.ig.com/gateway/deal'

#demo server
#url = 'https://demo-api.ig.com/gateway/deal'


s = requests.Session()
account = {'identifier': ig_account, 'password': pass_ig_account} 
s.headers = {'Content-Type': 'application/json; charset=UTF-8',
            'Accept': 'application/json; charset=UTF-8',
            'VERSION': '2',
            'X-IG-API-KEY': ig_api_key}

#use the 'headers' parameter to set the HTTP headers:
r = s.post(url + '/session', json = account)
s.headers.update({'X-SECURITY-TOKEN': r.headers['X-SECURITY-TOKEN'],'CST': r.headers['CST']})
s.headers.update({'Version': '2'})

#prod
#url = 'https://api.ig.com/gateway/deal/prices/IX.D.DAX.FWM2.IP/MINUTE_15/2021-07-27 17:00:00/2021-08-27 21:00:00'
url = 'https://api.ig.com/gateway/deal/prices/IX.D.DAX.IFS.IP/MINUTE_15/2021-09-13 14:00:00/2021-11-24 22:00:00'



#demo

#url = 'https://demo-api.ig.com/gateway/deal/prices/IX.D.DAX.FWM2.IP/MINUTE_15/2020-11-09 06:45:00/2020-12-17 12:00:00'
#url = 'https://demo-api.ig.com/gateway/deal/prices/IX.D.DAX.FWM2.IP/MINUTE_15/2020-12-15 10:15:00/2020-12-15 10:30:00'
#url = 'https://demo-api.ig.com/gateway/deal/prices/IX.D.DAX.FWM2.IP/MINUTE_15/2021-09-13 14:00:00/2021-11-24 22:00:00'

#url = 'https://demo-api.ig.com/gateway/deal/prices/IX.D.DAX.FWM2.IP'
#use the 'headers' parameter to set the HTTP headers:

resp_data = s.get(url)
print(resp_data.json())
resp_dict = json.loads(resp_data.text)
print(type(resp_dict))
resp_list = resp_dict["prices"]
print(type(resp_list))
hist_prices = list(resp_list)


In [ ]:
df = pd.DataFrame.from_dict(pd.json_normalize(hist_prices), orient='columns')
df.count()

In [ ]:
df

In [ ]:
df.to_csv (r'dax_M15_130921_241121.csv', index = False, header=True)

In [ ]:
import pandas as pd
dfa = pd.read_csv('dax_M15_010620_051020.csv')
dfb = pd.read_csv('dax_M15_051020_171220.csv')
dfc = pd.read_csv('dax_M15_171220_290121.csv')
dfd = pd.read_csv('dax_M15_290121_120321.csv')
dfe = pd.read_csv('dax_M15_140621_270721.csv')
dff = pd.read_csv('dax_M15_270721_270821.csv')
dfg = pd.read_csv('dax_M15_270821_130921.csv')
dfh = pd.read_csv('dax_M15_130921_241121.csv')


In [ ]:
#concatenate the periods into a common df
df = pd.concat([dfa, dfb, dfc, dfd, dfe, dff, dfg, dfh])

In [ ]:
# testing only
#df_1 = pd.concat([dfe, dff, dfg])

In [ ]:
df = df.reset_index(drop=True)
df.count()

In [ ]:
import numpy as np

df.drop('openPrice.lastTraded', inplace=True, axis=1)
df.drop('closePrice.lastTraded', inplace=True, axis=1)
df.drop('highPrice.lastTraded', inplace=True, axis=1)
df.drop('lowPrice.lastTraded', inplace=True, axis=1)
#df.drop('snapshotTime', inplace=True, axis=1)
#Calculate mean values
df.insert(1, 'openp', ((df['openPrice.ask'] + df['openPrice.bid'])/2).astype(float))
df.insert(1, 'closep', ((df['closePrice.ask'] + df['closePrice.bid'])/2).astype(float))
df.insert(1, 'highp', ((df['highPrice.ask'] + df['highPrice.bid'])/2).astype(float))
df.insert(1, 'lowp', ((df['lowPrice.ask'] + df['lowPrice.bid'])/2).astype(float))
df.insert(1, 'midp', ((df['closep'] + df['openp'])/2).astype(float))
df.rename(columns={'lastTradedVolume': 'volume'}, inplace=True)
df.drop('openPrice.ask', inplace=True, axis=1)
df.drop('closePrice.ask', inplace=True, axis=1)
df.drop('highPrice.ask', inplace=True, axis=1)
df.drop('lowPrice.ask', inplace=True, axis=1)
df.drop('openPrice.bid', inplace=True, axis=1)
df.drop('closePrice.bid', inplace=True, axis=1)
df.drop('highPrice.bid', inplace=True, axis=1)
df.drop('lowPrice.bid', inplace=True, axis=1)

#Add weighted moving averages
#wma10
weights = np.arange(1,11) #this creates an array with integers 1 to 10 included
wma10 = df['midp'].rolling(10).apply(lambda mid: np.dot(mid, weights)/weights.sum(), raw=True)
df.insert(1, 'wma10', (wma10).astype(float))
#wma20
weights = np.arange(1,21) #this creates an array with integers 1 to 10 included
wma20 = df['midp'].rolling(20).apply(lambda mid: np.dot(mid, weights)/weights.sum(), raw=True)
df.insert(1, 'wma20', (wma20).astype(float))
#wma50
weights = np.arange(1,51) #this creates an array with integers 1 to 10 included
wma50 = df['midp'].rolling(50).apply(lambda mid: np.dot(mid, weights)/weights.sum(), raw=True)
df.insert(1, 'wma50', (wma50).astype(float))
#wma100
weights = np.arange(1,101) #this creates an array with integers 1 to 10 included
wma100 = df['midp'].rolling(100).apply(lambda mid: np.dot(mid, weights)/weights.sum(), raw=True)
df.insert(1, 'wma100', (wma100).astype(float))
#wma200
weights = np.arange(1,201) #this creates an array with integers 1 to 10 included
wma200 = df['midp'].rolling(200).apply(lambda mid: np.dot(mid, weights)/weights.sum(), raw=True)
df.insert(1, 'wma200', (wma200).astype(float))
df = df.dropna()
df = df.reset_index(drop=True)
#df['snapshotTime'] = pd.to_datetime(df['snapshotTime'])

# Display
pd.reset_option('display.max_rows')
pd.options.display.max_rows = 1000
df.head(20)
#df.to_csv (r'prices_dataframe.csv', index = False, header=True)
print(df.count())

In [ ]:
#df.drop('midp', inplace=True, axis=1)
df.drop('midp', inplace=True, axis=1)

In [ ]:
def wwma(values, n):
    """
     J. Welles Wilder's EMA 
    """
    return values.ewm(alpha=1/n, adjust=False).mean()

def atr(df, n=14):
    data = df.copy()
    high = data['highp']
    low = data['lowp']
    close = data['closep']
    data['tr0'] = abs(high - low)
    data['tr1'] = abs(high - close.shift())
    data['tr2'] = abs(low - close.shift())
    tr = data[['tr0', 'tr1', 'tr2']].max(axis=1)
    atr = wwma(tr, n)
    return atr

In [ ]:
#Add Average True Range (14)
atr(df)
df['atr14']=atr(df)
print(type(df))
pd.options.display.max_rows = 1000

In [ ]:
df

In [ ]:
df.describe().transpose()

In [ ]:
import matplotlib.pyplot as plt
#from mpl_finance import candlestick_ohlc

plt.plot(np.array(df['openp'])[10:200], '>')
#plt.plot(np.array(df['closep'])[10:200], '<')
#plt.plot(np.array(df['closep'])[400:500], 'o')
#plt.plot(np.array(df['atr14'])[100:11500], 's')
#plt.plot(np.array(df['wma50'])[0:19])
#plt.plot(np.array(df['wma200'])[0:19])
#plt.plot(np.array(df['day_cos'])[:150])
#plt.plot(np.array(df['year_sin'])[:150])
#plt.plot(np.array(df['year_cos'])[:150])
plt.xlabel('Time [h]')
plt.title('Open price')

In [ ]:
#Uncomment the following lines to only use rows where volume > 200

df = df[df.volume > 200]
df = df.reset_index(drop=True)


In [ ]:
#Normalize date time. In this case we will only use the day part (based on wehter the chosen timeseries is e.g. 15min or Daily)
import datetime
import numpy as np

#date_time is set to timestamp integer value via the .astype('int64')
date_time = pd.to_datetime(df.pop('snapshotTime'),).astype('int64') // 10**9
#timestamp_s = date_time.map(datetime.datetime.timestamp)
day = 24*60*60
week = 7 * day
year = (365.2425)*day
df['day_sin'] = np.sin(date_time * (2 * np.pi / day))
df['week_sin'] = np.sin(date_time * (2 * np.pi / week))
#df['day_cos'] = np.cos(timestamp_s * (2 * np.pi / day))
#df['year_sin'] = np.sin(timestamp_s * (2 * np.pi / year))
#df['year_cos'] = np.cos(timestamp_s * (2 * np.pi / year))


In [ ]:
import matplotlib.pyplot as plt

plt.plot(np.array(df['day_sin'])[:401])
plt.plot(np.array(df['week_sin'])[:401])
#plt.plot(np.array(df['day_cos'])[:150])
#plt.plot(np.array(df['year_sin'])[:150])
#plt.plot(np.array(df['year_cos'])[:150])
plt.xlabel('Time [h]')
plt.title('Time of day signal')

In [ ]:
#DropExecute if dropping time signals
df.drop('day_sin', inplace=True, axis=1)
df.drop('week_sin', inplace=True, axis=1)

In [ ]:
#Build category labels by looping through dataset
#Hyperparameters for high and low factor and how manny steps into the future

f_look = 6
phigh = 1.5
plow = 1.5
i=0
category = []
row_count = len(df.index)
#print("for loop")
while i < row_count:
#    print(f'index: {i}')a
    hhcompare= df.iloc[i, df.columns.get_loc('closep')] + df.iloc[i, df.columns.get_loc('atr14')]*phigh
    llcompare= df.iloc[i, df.columns.get_loc('closep')] - df.iloc[i, df.columns.get_loc('atr14')]*phigh
    hlcompare= df.iloc[i, df.columns.get_loc('closep')] + df.iloc[i, df.columns.get_loc('atr14')]*plow
    lhcompare= df.iloc[i, df.columns.get_loc('closep')] - df.iloc[i, df.columns.get_loc('atr14')]*plow
#    print(hhcompare)
#    print(llcompare)
#    print(hlcompare)
#    print(lhcompare)
#    b= (df.loc[i:i+f_look, 'highp'].max() > hhcompare) and (df.loc[i:i+f_look, 'lowp'].min() > lhcompare)
#    c= (df.loc[i:i+f_look, 'lowp'].min() < llcompare) and (df.loc[i:i+f_look, 'highp'].max() < hlcompare)
    b= (df.loc[i:i+f_look, 'highp'].max() > hhcompare) and (df.loc[i:i+f_look, 'lowp'].min() > lhcompare)
    c= (df.loc[i:i+f_look, 'lowp'].min() < llcompare) and (df.loc[i:i+f_look, 'highp'].max() < hlcompare)
#    if not(b == c and not b):
#        if b :
#            b1= df.loc[i:i+10, 'highp'].idxmax()
#            print(f'bye is {b1-i}')
#            print(df.loc[i:i+10, 'highp'].max())
#        elif c :
#            c1= df.loc[i:i+10, 'lowp'].idxmin()
#            print(f'sell is {c1-i}')
#            print(df.loc[i:i+10, 'lowp'].min())
#     else :
#        print('hold')
    if (b == c and not b):
        a = True
    else:
        a = False
            
    category.append([int(a), int(b), int(c)])
    i +=1
        
pd.reset_option('display.max_rows')
pd.options.display.max_rows = 250
df_labels = pd.DataFrame(category, columns=["hold", "bye", "sell"])
df_labels.head(230)
  


In [ ]:
#Scaling
#First calculate the change in pips
from sklearn.preprocessing import minmax_scale
df['low_pip']=df.lowp-df.lowp.shift(1)
df = df.dropna()
df = df.reset_index(drop=True)



In [ ]:
df['lowp_z'] = zscore(df['low_pip'])
df

In [ ]:
#df['lowp_rtn'] = minmax_scale(df['lowp_rtn'], feature_range=(-1, 1))
#df['highp_minmax'] = minmax_scale(df['highp_rtn'], feature_range=(-1, 1))
#df['openp_rtn'] = minmax_scale(df['openp_rtn'], feature_range=(-1, 1))
#df['closep_rtn'] = minmax_scale(df['closep_rtn'], feature_range=(-1, 1))
#df['highp_z'] = zscore(df['highp_rtn'])
#df
from sklearn.preprocessing import RobustScaler
X = [[ 1., -2.,  2.],
     [ -2.,  1.,  3.],
     [ 4.,  1., -2.]]
transformer = RobustScaler().fit(X)
transformer
transformer.transform(X)

In [ ]:
#np_low = df.loc[:,'lowp_rtn']
#lowp_rtn_np =  np_low.values
mynp = df.loc[:,'low_pip'].values.reshape(-1, 1)
#lowp_rtn_np =  np_low.values
#print(lowp_rtn_np)
#mynp = lowp_rtn_np.reshape(-1, 1)
print(mynp)

transformer = RobustScaler().fit(mynp)
transformer
transformer.transform(mynp)

In [ ]:
df_mp = pd.DataFrame(mynp, columns=["low_adj"])
df_mp

In [ ]:
transformer
RobustScaler()
transformer.transform(X)
array([[ 0. , -2. ,  0. ],       [-1. ,  0. ,  0.4],       [ 1. ,  0. , -1.6]])
X

In [ ]:
df['vma10_diff'] = df['closep']-df['wma10']
df['vma20_diff'] = df['closep']-df['wma20']
df['vma50_diff'] = df['closep']-df['wma50']
df['vma100_diff'] = df['closep']-df['wma100']
df['vma200_diff'] = df['closep']-df['wma200']

df['vma10_diff'] = minmax_scale(df['vma10_diff'], feature_range=(-1, 1))
df['vma20_diff'] = minmax_scale(df['vma20_diff'], feature_range=(-1, 1))
df['vma50_diff'] = minmax_scale(df['vma50_diff'], feature_range=(-1, 1))
df['vma100_diff'] = minmax_scale(df['vma100_diff'], feature_range=(-1, 1))
df['vma200_diff'] = minmax_scale(df['vma200_diff'], feature_range=(-1, 1))

df

In [ ]:
import matplotlib.pyplot as plt
#print(df['lowp_z'].hist(bins=1000, figsize = (10,5)))
print(df_mp.hist(bins=1000, figsize = (10,5)))
#print(df['highp_minmax'].hist(bins=1000, figsize = (10,5)))
#print(df['highp_rtn'].hist(bins=1000, figsize = (10,5)))

In [ ]:
#import matplotlib.pyplot as plt
print(df['highp_z'].hist(bins=1000, figsize = (10,5)))
#print(df['highp_minmax'].hist(bins=1000, figsize = (10,5)))
#print(df['highp_rtn'].hist(bins=1000, figsize = (10,5)))

In [ ]:
#(df.highp_minmax > 78).sum()
print(df.loc[df['highp_minmax'] > 0.8])
#print(df.loc[df['snapshotTime'] == '2021/06/14 08:45:00'])
#print(df.loc[[10771]])

In [ ]:
df_mp.describe().transpose()

In [ ]:
# Standardize ranges with zscore
from scipy.stats import zscore
df['wma10'] = zscore(df['wma10'])
df['wma20'] = zscore(df['wma20'])
df['wma50'] = zscore(df['wma50'])
df['wma100'] = zscore(df['wma100'])
df['wma200'] = zscore(df['wma200'])
df['lowp'] = zscore(df['lowp'])
df['highp'] = zscore(df['highp'])
df['openp'] = zscore(df['openp'])
df['closep'] = zscore(df['closep'])
df['volume'] = zscore(df['volume'])
df['atr14'] = zscore(df['atr14'])


In [ ]:
#Combine the main dataframe and the created labels
df = pd.concat([df, df_labels], axis=1)
#drop NaN rows
df = df.dropna()
df = df.reset_index(drop=True)
df.count()

In [ ]:
df.describe().transpose()

In [ ]:
#df.hold.count(1)
print(df[df['hold']>0].sell.count())
print(df[df['sell']>0].bye.count())
print(df[df['bye']>0].hold.count())


In [ ]:
#Remove fraction of random "hold" rows to balance data
mysample = df.query('hold > 0').sample(frac=0.342).index
df.drop(mysample , inplace=True)
print(df[df['hold']>0].sell.count())


In [ ]:
df = df.reset_index(drop=True)
df.count()

In [ ]:
#df[['snapshotTime', 'hold', 'bye', 'sell']][17250:17500]



Split the data
We'll use a (70%, 20%, 10%) split for the training, validation, and test sets. Note the data is not being randomly shuffled before splitting. This is for two reasons.

It ensures that chopping the data into windows of consecutive samples is still possible.
It ensures that the validation/test results are more realistic, being evaluated on data collected after the model was trained.

In [ ]:
column_indices = {name: i for i, name in enumerate(df.columns)}

n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

num_features = df.shape[1]

In [ ]:
train_df.count()

In [ ]:
#Recreate the labels frames and drop the colums from the feature df

train_labels_df = train_df[['hold','bye', 'sell']]
val_labels_df = val_df[['hold','bye', 'sell']]
test_labels_df = test_df[['hold','bye', 'sell']]

In [ ]:
train_df.drop('hold', inplace=True, axis=1)
val_df.drop('hold', inplace=True, axis=1)
test_df.drop('hold', inplace=True, axis=1)
train_df.drop('bye', inplace=True, axis=1)
val_df.drop('bye', inplace=True, axis=1)
test_df.drop('bye', inplace=True, axis=1)
train_df.drop('sell', inplace=True, axis=1)
val_df.drop('sell', inplace=True, axis=1)
test_df.drop('sell', inplace=True, axis=1)
#df_labels.count()

In [ ]:
train_df.count()
#train_df.count()

In [ ]:
val_labels_df

Sequencing with Keras TimeseriesGenerator
Create the numpy arrays to be used by the keras models using the Keras TimeseriesGenerator

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np

lenght = 20

#training set
data_gen_train = TimeseriesGenerator(train_df.to_numpy(), train_labels_df.to_numpy(),
                               length=lenght, sampling_rate=1,
                               batch_size=15862)
batch_0 = data_gen_train[0]
train_np, train_labels_np = batch_0

#validation set
data_gen_val = TimeseriesGenerator(val_df.to_numpy(), val_labels_df.to_numpy(),
                               length=lenght, sampling_rate=1,
                               batch_size=15862)
batch_0 = data_gen_val[0]
val_np, val_labels_np = batch_0

#test set
data_gen_test = TimeseriesGenerator(test_df.to_numpy(), test_labels_df.to_numpy(),
                               length=lenght, sampling_rate=1,
                               batch_size=15862)
batch_0 = data_gen_test[0]
test_np, test_labels_np = batch_0



print(train_np.shape)
print(train_labels_np.shape)
print(val_np.shape)
print(val_labels_np.shape)
print(test_np.shape)
print(test_labels_np.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, LSTM
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

# Model configuration
batch_size = 6
img_width, img_height, img_num_channels = 13, 20, 1
loss_function = sparse_categorical_crossentropy
no_classes = 3
no_epochs = 50
optimizer = Adam()
validation_split = 0.2
verbosity = 1

# Determine shape of the data
input_shape = (img_width, img_height, img_num_channels)
train_np.shape[0]
if K.image_data_format() == 'channels_first':
    train_np = train_np.reshape(train_np.shape[0], 1, img_height, img_width)
    test_np = test_np.reshape(test_np.shape[0], 1, img_height, img_width)
    input_shape = (1, img_height, img_width)
else:
    train_np = train_np.reshape(train_np.shape[0], img_height, img_width, 1)
    test_np = test_np.reshape(test_np.shape[0], img_height, img_width, 1)
    input_shape = (img_height, img_width, 1)
 

print(train_np.shape)


In [ ]:
train_np.shape[0]

In [ ]:
# Parse numbers as floats
#input_train = input_train.astype('float32')
#input_test = input_test.astype('float32')

# Create the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='tanh'))
model.add(Dense(no_classes, activation='softmax'))

# Compile the model
#model.compile(loss=loss_function,
#              optimizer=optimizer,
#              metrics=['accuracy'])

model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])
# Fit data to model
#history = model.fit(train_np, train_labels_np,
#            batch_size=batch_size,
#            epochs=no_epochs,
#            verbose=verbosity,
#            validation_split=validation_split)

# Generate generalization metrics
#score = model.evaluate(test_np, test_labels_np, verbose=0)
#print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')




In [ ]:
# Parse numbers as floats
#input_train = input_train.astype('float32')
#input_test = input_test.astype('float32')

# Create the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='tanh', input_shape=input_shape))
model.add(Conv2D(64, kernel_size=(3, 3), activation='tanh'))
model.add(Conv2D(128, kernel_size=(3, 3), activation='tanh'))
model.add(Flatten())
model.add(Dense(128, activation='tanh'))
#model.add(Dense(64, activation='tanh'))
model.add(Dense(no_classes, activation='softmax'))

# Compile the model
#model.compile(loss=loss_function,
#              optimizer=optimizer,
#              metrics=['accuracy'])

model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])
# Fit data to model
#history = model.fit(train_np, train_labels_np,
#            batch_size=batch_size,
#            epochs=no_epochs,
#            verbose=verbosity,
#            validation_split=validation_split)

# Generate generalization metrics
#score = model.evaluate(test_np, test_labels_np, verbose=0)
#print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

model.summary()


In [ ]:
from tensorflow.keras.layers import TimeDistributed, Conv1D
model = Sequential()
#model.add(TimeDistributed(Conv1D(32, 3, padding='same', strides=2, activation='tanh'), input_shape = input_shape))
model.add(Conv2D(32, kernel_size=(3, 3), activation='tanh', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='tanh'))
#model.add(Conv2D(32, (3, 3), activation='tanh'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
#model.add(LSTM(100))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(units=64, return_sequences=True))
model.add(Flatten())
model.add(Dense(128, activation='tanh'))
#model.add(Dense(64, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(no_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])
model.summary()

In [ ]:
import tensorflow as tf
import time

start_time = time.time()

model.fit(train_np, train_labels_np,
          batch_size=batch_size,
          epochs=2,
          verbose=2,
          validation_data=(test_np, test_labels_np))
score = model.evaluate(test_np, test_labels_np, verbose=0)
print('Test loss: {}'.format(score[0]))
print('Test accuracy: {}'.format(score[1]))

elapsed_time = time.time() - start_time
#print("Elapsed time: {}".format(hms_string(elapsed_time)))

In [ ]:
val_np.shape

In [ ]:
# Predict using either GPU or CPU, send the entire dataset.  This might not work on the GPU.
# Set the desired TensorFlow output level for this example

val_np = val_np.reshape(val_np.shape[0], img_height, img_width, 1)
input_shape = (img_height, img_width, 1)
score = model.evaluate(val_np, val_labels_np, verbose=0)
print('Test loss: {}'.format(score[0]))
print('Test accuracy: {}'.format(score[1]))

In [ ]:
from sklearn import metrics

# For GPU just grab the first 100 images
small_x = val_np[810:1905]
small_y = val_labels_np[810:1905]
small_y2 = np.argmax(small_y,axis=1)
pred = model.predict(small_x)

#pred = np.argmax(pred,axis=1)
#score = metrics.accuracy_score(small_y2, pred)

print('Accuracy: {}'.format(score))
#print(pred)
#print(f"Shape: {pred.shape}")
print(pred[500:600])

In [ ]:

print(small_y[800:1300])

In [ ]:
print(small_x)